In [150]:

import torch 



In [151]:
input = torch.tensor([0,1,2])
input

tensor([0, 1, 2])

In [152]:
from torch.nn import Embedding

embedding = Embedding(3,4)
embedded_sentence =embedding(input)
embedded_sentence,embedded_sentence.shape


(tensor([[ 0.8202,  0.8283,  0.2457,  1.1370],
         [ 0.1938, -2.9892,  0.5100, -0.1527],
         [ 0.0823,  1.3379,  0.5307,  0.5900]], grad_fn=<EmbeddingBackward0>),
 torch.Size([3, 4]))

In [153]:

q_vector=[]
k_vector=[]
v_vector=[]
q_vector.clear()
k_vector.clear()
v_vector.clear()

Wq = torch.rand(4,3)
Wk = torch.rand(4,3)
Wv = torch.rand(4,3)


for word in embedded_sentence:
    q_vector.append(torch.matmul(word.unsqueeze(0),Wq))
    k_vector.append(torch.matmul(word.unsqueeze(0),Wk))
    v_vector.append(torch.matmul(word.unsqueeze(0),Wv))


In [154]:
q_vector

[tensor([[1.7329, 1.6018, 1.7664]], grad_fn=<MmBackward0>),
 tensor([[-1.9438, -0.9838, -1.2064]], grad_fn=<MmBackward0>),
 tensor([[1.3765, 1.0368, 1.5225]], grad_fn=<MmBackward0>)]

In [155]:
k_vector , k_vector[0].shape

([tensor([[0.9775, 1.6736, 1.0405]], grad_fn=<MmBackward0>),
  tensor([[-0.4639, -1.1571, -0.7377]], grad_fn=<MmBackward0>),
  tensor([[0.6633, 1.3730, 0.5662]], grad_fn=<MmBackward0>)],
 torch.Size([1, 3]))

In [156]:
v_vector

[tensor([[2.3624, 1.3323, 1.8101]], grad_fn=<MmBackward0>),
 tensor([[-2.3875, -1.7549, -2.4495]], grad_fn=<MmBackward0>),
 tensor([[2.2111, 1.4592, 1.7035]], grad_fn=<MmBackward0>)]